In [1]:
import pandas as pd
import fiona
from rdflib import Graph, Namespace, URIRef, XSD, Literal
from shapely.wkt import loads

### Parse GADM data and store relevant data in dictionary

In [2]:
# specify path to datasets
path_to_gadm_aut = "/mnt/data/GADM/RDF/gadm_AUT_RDF_population.ttl"
path_to_gadm_deu = "/mnt/data/GADM/RDF/gadm_DEU_RDF_population.ttl"

In [3]:
# create graph and bind namesspaces to it
gadm = Namespace("http://example.com/ontologies/gadm#")
graph = Graph()
graph.bind("gadm", gadm)
graph.parse(path_to_gadm_aut, format='ttl')
graph.parse(path_to_gadm_deu, format='ttl')

<Graph identifier=N7047c3a0414640b7a892afc1f17a68c1 (<class 'rdflib.graph.Graph'>)>

In [4]:
query = """
PREFIX geo: <http://www.opengis.net/ont/geosparql#> 

SELECT ?gadm_ent ?geom
WHERE {
    ?gadm_ent a gadm:AdministrativeUnit ;
        geo:hasGeometry ?geom_ent .
    ?geom_ent geo:asWKT ?geom .
}
"""

ref_grid_ls = []

# Execute the query and iterate over the results
for row in graph.query(query):
    gadm_ent, geom = row
    geom_wkt = loads(geom)
    cell_dict = {}
    cell_dict["gadm_ent"] = gadm_ent
    cell_dict["gadm_geom"] = geom_wkt
    ref_grid_ls.append(cell_dict)

### Enrich CAMS ref raster with GADM intersection information

In [10]:
path_to_cams_ref_raster_graph = "/mnt/data/CAMS/RDF/CAMS_reference_grid.ttl"

# create graph and bind namesspaces to it
aqqa = Namespace("http://example.com/ontologies/aqqa#")

graph = Graph()
graph.bind("aqqa", aqqa)

graph.parse(path_to_cams_ref_raster_graph, format='ttl')

<Graph identifier=N5fadb5a422b94a2e94ea783eae53751d (<class 'rdflib.graph.Graph'>)>

In [11]:
# iterate over all cell entities 
query = """
SELECT ?s ?geom
WHERE {
    ?s a sosa:FeatureOfInterest ;
       geo:hasGeometry ?geom_cell_ent .
    ?geom_cell_ent geo:asWKT ?geom .
}
"""

# Execute the query and iterate over the results
for i, row in enumerate(graph.query(query)):
    foi_ent, geom = row
    target_geom = loads(geom)

    
    # Iterate through the list of dictionaries
    for item in ref_grid_ls:
        gadm_uri = item["gadm_ent"]
        gadm_geom = item["gadm_geom"]

    
        # Check if the cell_geom intersects with the target_geom
        if target_geom.intersects(gadm_geom):
            graph.add((foi_ent, URIRef("http://www.opengis.net/ont/geosparql#intersects"), gadm_uri)) 
            #print(foi_ent, URIRef("http://www.opengis.net/ont/geosparql#intersects"), gadm_uri)


In [12]:
graph.serialize(destination="/mnt/data/CAMS/RDF/CAMS_reference_grid_gadm_connections.ttl", format="turtle")

<Graph identifier=N5fadb5a422b94a2e94ea783eae53751d (<class 'rdflib.graph.Graph'>)>